# Guidelines for Prompting
In this lesson, you'll practice two prompting principles and their related tactics in order to write effective prompts for large language models.

## Setup
#### Load the API key and relevant Python libaries.

In this course, we've provided some code that loads the OpenAI API key for you.

#### helper function
Throughout this course, we will use OpenAI's `gpt-3.5-turbo` model and the [chat completions endpoint](https://platform.openai.com/docs/guides/chat). 

This helper function will make it easier to use prompts and look at the generated outputs.  
**Note**: In June 2023, OpenAI updated gpt-3.5-turbo. The results you see in the notebook may be slightly different than those in the video. Some of the prompts have also been slightly modified to product the desired results.

In [2]:
import os
from dotenv import load_dotenv, find_dotenv
import google.generativeai as genai

_ = load_dotenv(find_dotenv()) # read local .env file
api_key  = os.getenv('GEMINI_API_KEY')

# Configure with your API key
genai.configure(api_key=api_key)

# Pick a Gemini model (text-only for now)
model = genai.GenerativeModel("gemini-2.5-flash")

## Prompting Principles
- **Principle 1: Write clear and specific instructions**
- **Principle 2: Give the model time to “think”**

### Tactics

#### Tactic 1: Use delimiters to clearly indicate distinct parts of the input
- Delimiters can be anything like:
    - \`\`\`
    - """
    - < >
    - `<tag> </tag>`
    - `:`

In [10]:
text = f"""
You should express what you want a model to do by
providing instructions that are as clear and
specific as you can possibly make them.
This will guide the model towards the desired output,
and reduce the chances of receiving irrelevant
or incorrect responses. Don't confuse writing a
clear prompt with writing a short prompt.
In many cases, longer prompts provide more clarity
and context for the model, which can lead to
more detailed and relevant outputs.
"""
prompt = f"""
Summarize the text delimited by triple backticks
into a single sentence.
```{text}```
"""
# Make the Gemini call
response = model.generate_content(prompt)
print(response.text)

To achieve desired and accurate AI model outputs, it is crucial to provide clear, specific, and often detailed instructions rather than just short ones, as this guides the model effectively.


#### Tactic 2: Ask for a structured output
- JSON, HTML

In [11]:
prompt = f"""
Generate a list of three made-up book titles along \
with their authors and genres. \
Provide them in JSON format with the following keys: \
book_id, title, author, genre. \
"""
response = model.generate_content(prompt)
print(response.text)

```json
[
  {
    "book_id": "BK001",
    "title": "The Chronos Curator",
    "author": "Dr. Aris Thorne",
    "genre": "Science Fiction"
  },
  {
    "book_id": "BK002",
    "title": "Whispers in the Silent Grove",
    "author": "Eleanor Vance",
    "genre": "Psychological Thriller"
  },
  {
    "book_id": "BK003",
    "title": "Aetherbloom's Lament",
    "author": "Lila Sterling",
    "genre": "Fantasy Romance"
  }
]
```


#### Tactic 3: Ask the model to check whether conditions are satisfied

In [12]:
text_1 = f"""
Making a cup of tea is easy! First, you need to get some
water boiling. While that's happening,
grab a cup and put a tea bag in it. Once the water is
hot enough, just pour it over the tea bag.
Let it sit for a bit so the tea can steep. After a
few minutes, take out the tea bag. If you
like, you can add some sugar or milk to taste.
And that's it! You've got yourself a delicious
cup of tea to enjoy.
"""
prompt = f"""
You will be provided with text delimited by triple quotes. 
If it contains a sequence of instructions,
re-write those instructions in the following format:

Step 1 - ...
Step 2 - …
…
Step N - …

If the text does not contain a sequence of instructions,
then simply write \"No steps provided.\"

\"\"\"{text_1}\"\"\"
"""
response = model.generate_content(prompt)
print("Completion for Text 1:")
print(response.text)

Completion for Text 1:
Step 1 - Get some water boiling.
Step 2 - Grab a cup and put a tea bag in it.
Step 3 - Pour the hot water over the tea bag.
Step 4 - Let it sit for a bit so the tea can steep.
Step 5 - After a few minutes, take out the tea bag.
Step 6 - If you like, add some sugar or milk to taste.


<mark>Initial reflections on differences on the model is that Gemini 2.5-flash is more verbose, it leaves explicit instructions in place when creating a list of steps. Example here leave in time measures and colloquial terms "sit for a bit".</mark>

In [13]:
text_2 = f"""
The sun is shining brightly today, and the birds are
singing. It's a beautiful day to go for a
walk in the park. The flowers are blooming, and the
trees are swaying gently in the breeze. People
are out and about, enjoying the lovely weather.
Some are having picnics, while others are playing
games or simply relaxing on the grass. It's a
perfect day to spend time outdoors and appreciate the
beauty of nature.
"""
prompt = f"""
You will be provided with text delimited by triple quotes. 
If it contains a sequence of instructions,
re-write those instructions in the following format:

Step 1 - ...
Step 2 - …
…
Step N - …

If the text does not contain a sequence of instructions,
then simply write \"No steps provided.\"

\"\"\"{text_2}\"\"\"
"""
response = model.generate_content(prompt)
print("Completion for Text 2:")
print(response.text)

Completion for Text 2:
No steps provided.


#### Tactic 4: "Few-shot" prompting

In [14]:
prompt = f"""
Your task is to answer in a consistent style.

<child>: Teach me about patience.

<grandparent>: The river that carves the deepest
valley flows from a modest spring; the
grandest symphony originates from a single note;
the most intricate tapestry begins with a solitary thread.

<child>: Teach me about ambivelance.
"""
response = model.generate_content(prompt)
print(response.text)

<grandparent>: The scent of woodsmoke, that can evoke both cozy warmth and the chill of departing summer;
the harmony, that binds together both the soaring major and the melancholic minor;
the bittersweet tale, that brings forth both a joyous memory and a quiet regret.


<mark>It was better at waxing poetic within the few-shot prompting.</mark>

### Principle 2: Give the model time to “think” 

#### Tactic 1: Specify the steps required to complete a task

In [15]:
text = f"""
In a charming village, siblings Jack and Jill set out on
a quest to fetch water from a hilltop
well. As they climbed, singing joyfully, misfortune
struck—Jack tripped on a stone and tumbled
down the hill, with Jill following suit. 
Though slightly battered, the pair returned home to
comforting embraces. Despite the mishap,
their adventurous spirits remained undimmed, and they
continued exploring with delight.
"""
# example 1
prompt_1 = f"""
Perform the following actions: 
1 - Summarize the following text delimited by triple
backticks with 1 sentence.
2 - Translate the summary into French.
3 - List each name in the French summary.
4 - Output a json object that contains the following
keys: french_summary, num_names.

Separate your answers with line breaks.

Text:
```{text}```
"""
response = model.generate_content(prompt_1)
print("Completion for prompt 1:")
print(response.text)

Completion for prompt 1:
Jack and Jill, siblings on a quest for water, tumbled down a hill but, though bruised, their adventurous spirits remained undimmed.

Jack et Jill, frère et sœur partis chercher de l'eau, ont dégringolé une colline mais, bien que contusionnés, leur esprit d'aventure est resté intact.

Jack
Jill

```json
{
  "french_summary": "Jack et Jill, frère et sœur partis chercher de l'eau, ont dégringolé une colline mais, bien que contusionnés, leur esprit d'aventure est resté intact.",
  "num_names": 2
}
```


#### Ask for output in a specified format

In [16]:
prompt_2 = f"""
Your task is to perform the following actions: 
1 - Summarize the following text delimited by 
  <> with 1 sentence.
2 - Translate the summary into French.
3 - List each name in the French summary.
4 - Output a json object that contains the 
  following keys: french_summary, num_names.

Use the following format:
Text: <text to summarize>
Summary: <summary>
Translation: <summary translation>
Names: <list of names in summary>
Output JSON: <json with summary and num_names>

Text: <{text}>
"""
response = model.generate_content(prompt_2)
print("\nCompletion for prompt 2:")
print(response.text)


Completion for prompt 2:
Text: <
In a charming village, siblings Jack and Jill set out on
a quest to fetch water from a hilltop
well. As they climbed, singing joyfully, misfortune
struck—Jack tripped on a stone and tumbled
down the hill, with Jill following suit. 
Though slightly battered, the pair returned home to
comforting embraces. Despite the mishap,
their adventurous spirits remained undimmed, and they
continued exploring with delight.
>
Summary: Siblings Jack and Jill tumbled down a hill while fetching water but remained adventurous despite their mishap.
Translation: Les frères et sœurs Jack et Jill sont tombés d'une colline en allant chercher de l'eau, mais sont restés aventureux malgré leur mésaventure.
Names: Jack, Jill
Output JSON: {"french_summary": "Les frères et sœurs Jack et Jill sont tombés d'une colline en allant chercher de l'eau, mais sont restés aventureux malgré leur mésaventure.", "num_names": 2}


#### Tactic 2: Instruct the model to work out its own solution before rushing to a conclusion

In [17]:
prompt = f"""
Determine if the student's solution is correct or not.

Question:
I'm building a solar power installation and I need
 help working out the financials. 
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost
me a flat $100k per year, and an additional $10 / square
foot
What is the total cost for the first year of operations 
as a function of the number of square feet.

Student's Solution:
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
"""
response = model.generate_content(prompt)
print(response.text)

The student's solution is **incorrect**.

Here's a breakdown of the error:

1.  **Land cost:** Correct (100x)
2.  **Solar panel cost:** Correct (250x)
3.  **Maintenance cost:** This is where the error occurs.
    *   The problem states: "flat $100k per year, and an additional $10 / square foot"
    *   The student wrote: "100,000 + 100x"
    *   The "additional $10 / square foot" should be `10x`, not `100x`. The student incorrectly used 100 instead of 10 for the per-square-foot maintenance cost.

**Corrected Solution:**

Let x be the size of the installation in square feet.

*   **Land cost:** 100x
*   **Solar panel cost:** 250x
*   **Maintenance cost:** 100,000 + 10x

**Total cost for the first year:**
100x + 250x + 100,000 + 10x
= (100 + 250 + 10)x + 100,000
= **360x + 100,000**


<mark>Gemini was better at spotting the incorrect student solution, I guess it already forces itself to use more time to reason through the problem first.</mark>

In [18]:
from IPython.display import display, Markdown, Latex, HTML, JSON
display(Markdown(response.text))

The student's solution is **incorrect**.

Here's a breakdown of the error:

1.  **Land cost:** Correct (100x)
2.  **Solar panel cost:** Correct (250x)
3.  **Maintenance cost:** This is where the error occurs.
    *   The problem states: "flat $100k per year, and an additional $10 / square foot"
    *   The student wrote: "100,000 + 100x"
    *   The "additional $10 / square foot" should be `10x`, not `100x`. The student incorrectly used 100 instead of 10 for the per-square-foot maintenance cost.

**Corrected Solution:**

Let x be the size of the installation in square feet.

*   **Land cost:** 100x
*   **Solar panel cost:** 250x
*   **Maintenance cost:** 100,000 + 10x

**Total cost for the first year:**
100x + 250x + 100,000 + 10x
= (100 + 250 + 10)x + 100,000
= **360x + 100,000**

#### Note that the student's solution is actually not correct.
#### We can fix this by instructing the model to work out its own solution first.

In [19]:
prompt = f"""
Your task is to determine if the student's solution
is correct or not.
To solve the problem do the following:
- First, work out your own solution to the problem including the final total. 
- Then compare your solution to the student's solution
and evaluate if the student's solution is correct or not. 
Don't decide if the student's solution is correct until 
you have done the problem yourself.

Use the following format:
Question:
```
question here
```
Student's solution:
```
student's solution here
```
Actual solution:
```
steps to work out the solution and your solution here
```
Is the student's solution the same as actual solution just calculated:
```
yes or no
```
Student grade:
```
correct or incorrect
```
Discrepancy between solutions:
```
if the student's solution is incorrect explain where and why

Question:
```
I'm building a solar power installation and I need help
working out the financials. 
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost me 
a flat $100k per year, and an additional $10 / square foot
What is the total cost for the first year of operations as 
a function of the number of square feet.
``` 
Student's solution:
```
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
```
Actual solution:
"""
response = model.generate_content(prompt)
display(Markdown(response.text))

Actual solution:
Let `x` be the size of the installation in square feet.

The total cost for the first year includes:
1.  **Land cost:** This is a one-time capital cost.
    Cost = $100/square foot * `x` square feet = `100x`
2.  **Solar panel cost:** This is a one-time capital cost.
    Cost = $250/square foot * `x` square feet = `250x`
3.  **Maintenance cost for the first year:** This is an annual operational cost.
    Flat fee = $100,000 per year
    Additional cost = $10/square foot * `x` square feet = `10x`
    Total maintenance cost for the first year = `100,000 + 10x`

Total cost for the first year of operations = (Land cost) + (Solar panel cost) + (First year's maintenance cost)
Total cost = `100x + 250x + (100,000 + 10x)`
Total cost = `100x + 250x + 10x + 100,000`
Total cost = `(100 + 250 + 10)x + 100,000`
Total cost = `360x + 100,000`

Is the student's solution the same as actual solution just calculated:
no

Student grade:
incorrect

Discrepancy between solutions:
The student made an error in calculating the maintenance cost. The problem states "an additional $10 / square foot" for maintenance, but the student used `$100 / square foot` in their calculation, resulting in `100x` for the variable part of the maintenance cost instead of the correct `10x`. This led to an incorrect coefficient for `x` in the final total cost function.

## Model Limitations: Hallucinations
- Boie is a real company, the product name is not real.

In [21]:
prompt = f"""
Tell me about Tooth Destroyer Hardcore EnamelGrinder UltraSlow Smart Toothbrush by Boie
"""
response = model.generate_content(prompt)
print(response.text)

It sounds like you've come across a very **humorous and definitely fictional** product name!

There is no actual toothbrush called the "Tooth Destroyer Hardcore EnamelGrinder UltraSlow Smart Toothbrush" by Boie (or anyone else, for that matter!). The name is clearly designed to be an ironic and exaggerated take on toothbrush marketing, especially with terms like "Tooth Destroyer" and "EnamelGrinder," which are the *opposite* of what a toothbrush aims to do, and "UltraSlow" which contradicts common electric toothbrush features.

**However, Boie is a real company!**

**Boie USA** (often just Boie) is known for making innovative toothbrushes, primarily recognized for:

1.  **Silicone Bristles:** Unlike traditional nylon bristles, Boie toothbrushes feature soft, hypoallergenic silicone bristles. This design is touted for being:
    *   **Gentle on gums and enamel:** They aim for effective cleaning without being abrasive.
    *   **Hygienic:** Silicone is less porous than nylon, which is cl

<mark>Caught the satire, perhaps it performed an internet search for the product also. Something to check later.</mark>
Let's try something else.

In [23]:
prompt = f"""
Tell me about AeroGlide UltraSlim Smart Toothbrush by Boie
"""
response = model.generate_content(prompt)
print(response.text)

It seems there might be a slight mix-up in the product name or brand you're recalling!

**I cannot find any product called the "AeroGlide UltraSlim Smart Toothbrush" specifically from the brand Boie.**

**Boie (often known as Boie USA)** is actually famous for its **manual toothbrushes**, which are known for their incredibly soft, rubber-like bristles (made from medical-grade thermoplastic elastomer) and minimalist design. They are designed for gentle, effective cleaning without any electronic or "smart" features.

Their flagship product is simply called the **Boie USA Toothbrush** (or similar variations), and it's a completely manual brush. It doesn't have app connectivity, sonic vibrations, or the kind of "smart" features implied by "Smart Toothbrush."

**Could you be thinking of a different brand, or perhaps a different product name?**

Brands like **Philips Sonicare, Oral-B, Quip, Burst, or Oclean** are well-known for offering "smart" electric toothbrushes that often feature slim d

<mark>Gemini shows it checks online for proof, this could fail when there is also misinformation to be found out there.</mark>

## Try experimenting on your own!

# Prompt Development Process

**Why:** Developing effective prompts is an iterative process, focusing on refining ideas rather than achieving perfection on the first try.

**How:** 
1. Start with an idea
2. Write a prompt
3. Run it
4. Analyze the result
5. Refine the prompt based on feedback
6. Iterate until achieving the desired outcome

**What:** The lesson demonstrates the process of developing prompts for applications using large language models, emphasizing the importance of clarity, specificity, and iteration in prompt engineering.

## Cheatsheet:
- Start with an idea for the task you want to achieve.
  - *Reasoning:* Clear objectives lead to focused prompts.
  - *Example:* Define the goal of summarizing a fact sheet for a chair.
- Write a clear and specific prompt.
  - *Reasoning:* Specific instructions guide the model effectively.
  - *Example:* Task the model with creating a product description based on the fact sheet.
- Analyze the result and refine the prompt iteratively.
  